In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="darkgrid")
from time import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
# Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA    

# Pretty display for notebooks
%matplotlib inline

In [ ]:
#without accuracy ---> 2
#SVM 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%pylab inline
from random import *
from subprocess import check_output
def Propublica_svm(X,Y):
    #Split data into training and test datasets (training will be based on 70% of data)
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0,shuffle=True) 
    #test_size: if integer, number of examples into test dataset; if between 0.0 and 1.0, means proportion
    print('There are {} samples in the training set and {} samples in the test set'.format(X_train.shape[0], X_test.shape[0]))

    
    #Scaling data
    #from sklearn.preprocessing import StandardScaler
    
    from sklearn.model_selection import cross_val_score

    #sc = StandardScaler(with_mean=False)
    
    
    #sc.fit(X_train)
    #X_train_std = sc.transform(X_train)
    #X_test_std = sc.transform(X_test)

    #X_train_std and X_test_std are the scaled datasets to be used in algorithms

    #Applying SVC (Support Vector Classification)
    from sklearn.svm import SVC
    svm = SVC(kernel='rbf', random_state=1, gamma=.1, C=10.0)
    svm.fit(X_train, Y_train)
    print('The accuracy of the SVM classifier on training data is {:.2f}'.format(svm.score(X_train, Y_train)))
    print('The accuracy of the SVM classifier on test data is {:.2f}'.format(svm.score(X_test, Y_test)))
    print('####Train prediction Label###############################################')
    Y_train_pred=svm.predict(X_train)
    #print(y_1)
    Y_test_pred=svm.predict(X_test)

    print('####Actual Train Label###############################################')


    print('####Change to colors###############################################')
        
    
    return X_test,Y_test_pred,Y_test
     



In [ ]:
#without accuracy ---> LPC
def main(datax, y_test, y_test_pred): 
        
    n=datax.shape[1]
    s=datax.shape[0]    
    data = np.zeros((s, n), dtype = int)
    
    r = np.zeros(n, dtype = int) 
    
    for i in range(n):
        if int(y_test.iloc[i])==1 :
            r[i]=1
        else :
            r[i]= -1  
    
    r2 = np.zeros(n, dtype = int) 
    for i in range(n):
        if int(y_test_pred[i])==1 :
            r2[i]=1
        else :
            r2[i]= -1          
    ar=[]
    
    for j in range(s):
        print("sensitive attribute ",(j+1)) 
        a=0
        b=0
        acc1=0
        acc2=0
        for i in range(n):
                data[j][i]= datax.iloc[j,i]
                if data[j][i]== 1 :
                    a=a+1
                    if r[i]==1:
                         acc1=acc1+1 

        print("total ,fair accepted, aceeptance rate:")             
        a1=float(acc1/a)
        print(a)
        
        print(acc1)
        print(a1)
        ar.append(a1)
        
    maxi= max(ar)
    mini= min(ar)
    DP=float(maxi-mini)
    print("data acceptance rates")
    print(ar)
    print("data DP")
    print(DP)                       

#use gamma epsilon for feasilbility check
    delta=1
    gama=[.2,.25,.3,.35,.4,.45,.5,.54]
    epsilon=[.01,.02,.05,.1,.15,.20,.25]
    fi= np.zeros(n,dtype=int) 
#     for delta in delta1:
    for gamma in gama:
        for eps in epsilon:
            u1,u2=min_max_lpc(data,gamma,eps,r2,delta)
            #######################Disp_impact#######################  
            print("gamma-epsilon-delta",gamma,eps,delta)
            accu_all=[]
            DP_all=[]
            precision_all=[]
            recall_all=[]
            acceptance_rate=np.zeros((7,28),dtype=float)
            count=0
#                 for alpha in np.arange(0,1.05,0.05):
#                     print("alpha: ",alpha)
#                     for i in range(n):

#                         z=random()
#                         if z < alpha:
#                                fi[i]= u1[i] 

#                         else:
#                                fi[i]= r2[i]

            for i in range(n):
                 fi[i] = u1[i]
            ar=[]

            for j in range(s):
#                         print("sensitive attribute ",(j+1)) 
                a=0
                b=0
                acc1=0
                acc2=0
                for i in range(n):
                        if data[j][i]== 1 :
                            a=a+1
                            if fi[i]==1:
                                 acc1=acc1+1 

#                         print("total ,fair accepted, aceeptance rate:")             
                a1=float(acc1/a)
                acceptance_rate[j][count]=float(a1)
                

#                         print(a)
#                         print(acc1)
#                         print(a1)
                ar.append(a1)
  
            count = count+1
            maxi=max(ar)
            mini= min(ar)
            DP=float(maxi-mini)
            print("acceptance rates")
            print(ar)
            print("DP")
            print(DP)
            f_acc=0
            for i in range(n):
                 if fi[i] == r[i]:
                        f_acc=f_acc+1
            f_acc_l=float((f_acc*100)/n) 
            
 #######################################################################33   
           
#                         print("sensitive attribute ",(j+1)) 
            TP=0
            FP=0
            FN=0
            precision=0
            recall=0
            for i in range(n):
                    if fi[i]==1 and r[i]==1:
                        TP=TP+1
                    if fi[i]==1 and r[i]==-1:
                        FP=FP+1 
                    if fi[i]==-1 and r[i]==1:
                        FN=FN+1
            precision=float(TP/(TP+FP))
            print(precision)
            recall=float(TP/(TP+FN))
            print(recall)
#                         print("total ,fair accepted, aceeptance rate:")             
            a1=float(acc1/a)
            acceptance_rate[j][count]=float(a1)
                

#                         print(a)
#                         print(acc1)
#                         print(a1)
              
    
 ###############################################################################################             
#                     print("accuracy : ",f_acc_l)
            
            precision_all.append(precision)
            recall_all.append(recall)
            accu_all.append(f_acc_l)
            DP_all.append(DP)
            print("precision", precision_all)
            print("recall", recall_all)
            print("final: acc", accu_all)
            print("final: dp", DP_all)
            
    alpha_weight=np.arange(0,1.05,.05)        
    return accu_all,DP_all,acceptance_rate,alpha_weight

In [ ]:

import time
import pulp as p 
def min_max_lpc(data1,gamma,eps,r,delta):
    import pulp as p 
    
    m=data1.shape[0]
    n=data1.shape[1]
    print('dimension of data')
    print(m,n)
    Lp_prob = p.LpProblem('Problem', p.LpMaximize)  
   
    
#     X=np.zeros(n+1,dtype=p.LpVariable)
    X=np.zeros(n+1,dtype=p.LpVariable)
    sizes=np.zeros(m,dtype=int)
    for i in range(m):
        count=0
        for j in range(n):
            if data1[i][j]==1:
                count=count+1               
        sizes[i]=count
  

    for i in range(n):
        var1=str(i)       
        X[i]=p.LpVariable(var1,lowBound=0,upBound=1,cat='Integer')
    
    X[n]=p.LpVariable(str(n),lowBound=0,upBound=1,cat='Continuous')   
        
#     X[n]=  p.LpVariable("z1",lowBound=0)
    #X[n+1]=  p.LpVariable("z2",lowBound=0)


    #########objective function#####################
#     Lp_prob += X[n] 
            
    Lp_prob += X[n]


    ##############constraint#################
    for i in range(2*m):
        if i<m:
#             Lp_prob += X[n] >= p.lpSum([2*(X[j]-0.5)*data1[i][j] for j in range(n)])
            Lp_prob += p.lpSum([2*(X[j]-0.5)*data1[i][j] for j in range(n)]) >= (2*gamma-1)*sizes[i]
            Lp_prob += p.lpSum([2*(X[j]-0.5)*data1[i][j] for j in range(n)]) <= ((2*gamma-1)+eps)*sizes[i]
            
 
         
    Lp_prob += p.lpSum([2*(X[i]-0.5)*r[i] for i in range(n)]) >= X[n]*n
    #n is the number of elements in sensitive attribute 
                 
#     Lp_prob += X[n] <= 42000
    
    #####################################
    status = Lp_prob.solve()   # Solver 
    print(p.LpStatus[status]) 
    print("discripency is:")        
    print(p.value(Lp_prob.objective))
    
    x=np.zeros(n,dtype=float)

   # The solution status 
    Synth1={}
    Synth2={}
    # # Printing the final solution 
    for i in range(n):
        if(p.value(X[i])==1):
            Synth1[i]=1 
            Synth2[i]=-1
        else:
            Synth1[i]=-1
            Synth2[i]=1
    Synthu1=Synth1  
    Synthu2=Synth2  
    
              
    return Synthu1,Synthu2   


In [ ]:
# without accuracy
import time
# import pulp as p 
# from random import *
data= pd.read_csv('data/propublica/compass.csv', skipinitialspace=True)
# data = data1[data1["race"].isin(["african-american", "caucasian"])]

print(data['African_American'].value_counts())
print(data['Female'].value_counts())
# print(data.shape[0],data.shape[1])
data=data.drop(columns=['id'])
# print(data.head())
# Age_Above_FourtyFive,Age_Below_TwentyFive, African_American,Female,  Two_yr_Recidivism  



data_c = data.drop(columns=[ 'Two_yr_Recidivism' ])
# print(sens)
print(data_c.head())
r=data[['Two_yr_Recidivism']]

X_test,Y_test_pred,Y_test = Propublica_svm(data_c , r)

X_test.reset_index(drop=True, inplace=True)
# Y_test_pred.reset_index()

Y_test.reset_index(drop=True, inplace=True)

print(X_test)
print(Y_test_pred)
print(Y_test)
sens=X_test[['African_American']]
print(sens)
p=sens.shape[0]


sens1 = pd.get_dummies(sens, columns=['African_American'], prefix =['african_american'])
sensitive = sens1.T
print(sensitive)


# (unique, counts) = numpy.unique(Y_test_pred, return_counts=True)
# frequencies = numpy.asarray((unique, counts)).T
# print(frequencies)
# print(r['y'].value_counts())

accu_all,DP_all,acceptance_rate,alpha_weight = main(sensitive, Y_test, Y_test_pred )

# for i in range(0,p):
#     if sens.loc[i,"12"]>25 :
#                sens.loc[i,"12"] = 1 
#     else :
#                sens.loc[i,"12"] = 2        
#     if r.loc[i,'20'] == 1 :
#                r.loc[i,"20"] = 1 
#     else: 
#                r.loc[i,"20"] = 0  

In [ ]:
accu_all,DP_all,acceptance_rate,alpha_weight = main(sensitive, Y_test, Y_test_pred )
#optimality check for each epsilons